In [ ]:
!kaggle datasets download -p datasets -d nadinpethiyagoda/vehicle-dataset-for-yolo --unzip

import super_gradients as sg
import torchinfo
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_ARCH = sg.common.object_names.Models.YOLO_NAS_S
CHECKPOINT_DIR = 'checkpoints'
EXPERIMENT_NAME = 'yolo-nas'

EPOCHS = 10
BATCH_SIZE = 16
WORKERS = 8 # Set equal to the CPU core count
RESUME = True # Resume training from the latest checkpoint

In [ ]:
'''
# Inference on an image using pre-trained weights
# Model Zoo - https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/model_zoo.md
model = sg.training.models.get(MODEL_ARCH, pretrained_weights='coco').to(DEVICE)

torchinfo.summary(model=model, input_size=(16, 3, 640, 640),
    col_names=['input_size', 'output_size', 'num_params', 'trainable'],
    col_width=20,
    row_settings=['var_names']
)

model.predict('https://en.wikipedia.org/wiki/File:401_Gridlock.jpg', conf=0.25).show()

# Inference on a video using pre-trained weights
INPUT_VIDEO_PATH = 'test.mp4'
OUTPUT_VIDEO_PATH = 'detection.mp4'
model.predict(INPUT_VIDEO_PATH).save(OUTPUT_VIDEO_PATH)
'''

In [ ]:
dataset_params = {
    'data_dir': 'datasets/vehicle dataset',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'valid/images',
    'test_labels_dir':'valid/labels',
    'classes': ['car', 'threewheel', 'bus', 'truck', 'motorbike', 'van']
}

num_classes = len(dataset_params['classes'])

train_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

val_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

test_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

print('Data augmentation:')
print(train_data.dataset.transforms)
print('Dataloader parameters:')
print(train_data.dataloader_params)
print('Dataset parameters')
print(train_data.dataset.dataset_params)

In [ ]:
train_data.dataset.plot(plot_transformed_data=False)

In [ ]:
train_data.dataset.plot(plot_transformed_data=True)

In [ ]:
# Remove DetectionMixup augmentation as can make the dataset to be difficult for models to perceive.
train_data.dataset.transforms.pop(2)
train_data.dataset.plot(plot_transformed_data=True)

In [ ]:
# Fine-tuning a pre-trained model
model = sg.training.models.get(MODEL_ARCH, num_classes=num_classes, pretrained_weights='coco').to(DEVICE)
trainer = sg.training.Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

train_params = {}
train_params['resume'] = RESUME
train_params['silent_mode'] = False
train_params['average_best_models'] = True
train_params['initial_lr'] = 5e-4
train_params['lr_mode'] = 'cosine'
train_params['cosine_final_lr_ratio'] = 0.1
train_params['optimizer'] = 'Adam'
train_params['optimizer_params'] = {'weight_decay': 1e-4}
train_params['max_epochs'] = EPOCHS
train_params['loss'] = sg.training.losses.PPYoloELoss(use_static_assigner=False, num_classes=num_classes, reg_max=16)
train_params['metric_to_watch'] = 'mAP@0.50:0.95' # Metric in which to record and save the best model (ckpt_best.pth)

# Validation metrics to log and display in Tensorboard
train_params['valid_metrics_list'] = [
        sg.training.metrics.DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=num_classes,
            normalize_targets=True,
            post_prediction_callback=sg.training.models.detection_models.pp_yolo_e.PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        ),

        sg.training.metrics.DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=num_classes,
            normalize_targets=True,
            post_prediction_callback=sg.training.models.detection_models.pp_yolo_e.PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ]

# tensorboard --logdir {CHECKPOINT_DIR}/{EXPERIMENT_NAME}
trainer.train(model=model, training_params=train_params, train_loader=train_data, valid_loader=val_data)

In [ ]:
# The super-gradients API saves three different checkpoints. One for the best model, one for the latest model, and one for the
# averaged weights of the 10 best models according to metric_to_watch.

# Testing the best model on the test set
sg.training.utils.checkpoint_utils.load_checkpoint_to_model(net=model, checkpoint_local_path=f'{CHECKPOINT_DIR}/{EXPERIMENT_NAME}/ckpt_best.pth')
# If an instance of model does not already exist - model = sg.training.models.get(MODEL_ARCH, num_classes=num_classes, checkpoint_path=f'{CHECKPOINT_DIR}/{EXPERIMENT_NAME}/ckpt_best.pth').to(DEVICE)

test_metrics = [
    sg.training.metrics.DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=num_classes,
            normalize_targets=True,
            post_prediction_callback=sg.training.models.detection_models.pp_yolo_e.PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ]

trainer.test(model=model, test_loader=test_data, test_metrics_list=test_metrics)

In [ ]:
# Testing the best model a web image
model.predict('https://en.wikipedia.org/wiki/File:401_Gridlock.jpg', conf=0.25).show(box_thickness=2, show_confidence=True)

In [ ]:
# Export the model to ONNX format
sg.training.models.conversion.convert_to_onnx(model, '.')